In [3]:
## importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

def accuracy():
    ## this function tells accuaracy of the current model on training set.
    count = 0
    for i in range(m):
        xx = x[:,i]
        yy = y[:,i]
        prediction_array = net.feedforward(xx.reshape(784,1))
        ind = np.argmax(prediction_array)
        if(ind == yy):
            count +=1
    print("Accuracy : ",(count/m)*100)


## importing data_frame from the CSV file.
data_frame = pd.read_csv(r'C:\Users\Anant Agarwal\Desktop\Machine Learning\Codes\8 - FeedForward Neural Network\train_set_full_shuffled.csv')
x = data_frame.iloc[:,1:].values ## shape is (m,n) 
y = data_frame.iloc[:,0].values  ## shape is (m,1) 

## number of examples
m = x.shape[0]                  
epochs = 12

## learning_rate
alpha = .5
x = x.T                          ## shape is (n,m) 
x = x/255                        ## normalizing the data
y = y.reshape(-1,1)
y = y.T                          ## shape is (1,m) 

## stochastic gradient descent
batch_size = 1

## defining sigmoid function
def sigmoid(z):
    return 1/(1+np.exp(-z))

## definining sigmoid gradient function "sigmoid prime"
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    ## class constructor
    def __init__(self,sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        np.random.seed(5)
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])]
    
    
    ## feedforward function
    def feedforward(self, a):
        for b,w in zip(self.biases,self.weights):
            a = sigmoid(np.dot(w,a) +b)
        return a
    
    ## Applying Mini_Batch Gradient Descent
    def gradient_descent(self,x,y,epochs,alpha):
        for j in range(epochs):
            print("Epoch : ", j+1)
            ## real_batch_size is a variable defined to take care of the unsurity of the
            ## size of the last batch.
            real_batch_size = batch_size
            ## loop for distributing batches
            for i in range(0,m,batch_size):
                ## xx and yy contains x's and y's for a particular batch
                xx, yy = x[:,i : (i+batch_size)], y[:,i : (i+batch_size)]
                
                ## initialization of weights and biases for a single batch
                nabla_b = [np.zeros(b.shape) for b in self.biases]
                nabla_w = [np.zeros(w.shape) for w in self.weights]
                
                ## implementation detail
                if(m-i<batch_size):
                    real_batch_size = m-i
                
                ## for each example, calculating the delta values and keeping it in the nabla variable
                for kk in range(real_batch_size):
                     ## finding out delta values by calling the backpropagation function
                    delta_nabla_b, delta_nabla_w = self.backprop(xx[:,kk],yy[:,kk])
                    
                    ## adding the delta_nabla values to nabla variables
                    nabla_b = [nb+dnb for nb,dnb in zip(nabla_b,delta_nabla_b)]
                    nabla_w = [nw+dnw for nw,dnw in zip(nabla_w,delta_nabla_w)]
                
                ## after each batch using the nabla values to change the weights
                self.weights = [w-(alpha/batch_size)*nw for w,nw in zip(self.weights, delta_nabla_w)]
                self.biases = [b-(alpha/batch_size)*nb for b,nb in zip(self.biases, delta_nabla_b)]
                
            ## calling out accuracy() function after each epoch to make sure our model is really working
            accuracy()
            
    def backprop(self,x,y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        ## feedforward propagation
        x = x.reshape(-1,1)
        activation = x
        activations = [x]
        zs = []
        
        for b,w in zip(self.biases,self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        ## backward propagation
        delta = self.cost_derivative(activations[-1], y)*sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)
        
        for layer in range(2, self.num_layers):
            delta = np.dot(self.weights[-layer+1].transpose(), delta) * sigmoid_prime(zs[-layer])
            nabla_b[-layer] = delta
            nabla_w[-layer] = np.dot(delta, activations[-layer-1].transpose())
        return (nabla_b, nabla_w)
    
    ## derivative of the cost function
    def cost_derivative(self,output_activations,y):
        target = np.zeros((10,1))
        target[y] = 1
        return 2*(output_activations - target)
        
## making a neural net with one hidden layer with 30 neurons    
net = Network([784,30,10])
## calling gradient descent to start the training process
net.gradient_descent(x,y,epochs,alpha)

Epoch :  1
Accuracy :  90.8547619047619
Epoch :  2
Accuracy :  92.0
Epoch :  3
Accuracy :  91.99761904761905
Epoch :  4
Accuracy :  92.93333333333334
Epoch :  5
Accuracy :  93.20714285714286
Epoch :  6
Accuracy :  93.81904761904762
Epoch :  7
Accuracy :  93.94761904761904
Epoch :  8
Accuracy :  93.78571428571428
Epoch :  9
Accuracy :  93.8952380952381
Epoch :  10
Accuracy :  93.78571428571428
Epoch :  11
Accuracy :  94.72142857142856
Epoch :  12
Accuracy :  95.02142857142857


In [4]:
## calling this code will generate predictions for the images in the test set which we'll eventually store in a CSV file.

test_set = pd.read_csv(r'C:\Users\Anant Agarwal\Desktop\Machine Learning\Kaggle\MNIST\Data\test.csv')
x_test = test_set.values ## shape is (m,n) i.e. (28000,784)
x_test = x_test/255
m_test = x_test.shape[0]
arr = []

for i in range(m_test):
    xxx = x_test[i,:].reshape(-1,1)
    result = np.argmax(net.feedforward(xxx))
    arr.append(result)

## making a dataframe
ddf = pd.DataFrame({'ImageId':[i+1 for i in range(m_test)],'label':arr})
## saving that dataframe as a CSV file
ddf.to_csv(r'C:\Users\Anant Agarwal\Desktop\Machine Learning\Codes\8 - FeedForward Neural Network\kagle_submission.csv',index=False)